In [15]:
%load_ext autoreload
%autoreload 2

In [143]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import glob
import tqdm
from scipy.stats import pearsonr as pcorr
import itertools
import re
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
# from src.features import metric_exploration

In [3]:
PATH_ROOT = "/home/shaul/workspace/GitHub/SOTA/"

In [4]:
cd {PATH_ROOT}

/home/shaul/workspace/GitHub/SOTA


In [166]:
df = pd.read_csv('/home/shaul/workspace/GitHub/SOTA/data/full_DS/full_metrics.csv', index_col= 0)
with open('/home/shaul/workspace/GitHub/SOTA/data/other/ba_all.txt','r+') as f:
    list_ba = f.read().splitlines() 

In [94]:
# # Need to change 'is_radical' and 'is_centrist' into one category
# df['radical_or_centralist'] = ['radical'if row['is_radical'] ==True else "centralist"  if row['is_centralist'] == True else "neither" for index,row in df.iterrows()]
# df.drop(columns=['is_radical','is_centralist'], inplace=True)

In [167]:
non_metric_columns = ['text1','text2','label','dataset','random','duration','total_seconds','pair_id','reduced_label','annotator','radical','radical_random','radical_non_random','radical_or_centralist','num_labels','bad_annotator']
categories = ['dataset', 'random','radical_or_centralist']

In [168]:
df.columns

Index([&#39;annotator&#39;, &#39;text1&#39;, &#39;text2&#39;, &#39;label&#39;, &#39;dataset&#39;, &#39;random&#39;, &#39;duration&#39;,
       &#39;total_seconds&#39;, &#39;pair_id&#39;, &#39;reduced_label&#39;, &#39;bleu&#39;, &#39;bleu1&#39;,
       &#39;glove_cosine&#39;, &#39;fasttext_cosine&#39;, &#39;BertScore&#39;, &#39;chrfScore&#39;,
       &#39;POS Dist score&#39;, &#39;1-gram_overlap&#39;, &#39;ROUGE-1&#39;, &#39;ROUGE-2&#39;, &#39;ROUGE-l&#39;,
       &#39;radical&#39;, &#39;radical_random&#39;, &#39;radical_non_random&#39;, &#39;num_labels&#39;,
       &#39;bad_annotator&#39;, &#39;L2_score&#39;, &#39;WMD&#39;, &#39;radical_or_centralist&#39;],
      dtype=&#39;object&#39;)

In [169]:
from src import metric_exploration
test = metric_exploration.Metrics_Corr(df2,non_metric_columns,categories)

In [170]:
example = test.compare_correlations(list_ba)

In [171]:
example['label_by_random']

,bleu,bleu1,glove_cosine,fasttext_cosine,BertScore,chrfScore,POS Dist score,1-gram_overlap,ROUGE-1,ROUGE-2,ROUGE-l,L2_score,WMD
0,0.006192,0.036388,-0.025129,-0.038641,0.029053,0.029845,0.002567,0.023812,0.032359,0.026236,0.036233,-0.040784,-0.027279
1,-0.017561,-0.045880,0.038510,0.041842,0.012594,-0.040251,-0.008199,-0.043527,-0.040694,-0.028073,-0.031789,0.028917,0.002130


### Look at the Non-Linear and Linear Models

In [273]:
from src import metric_exploration

test = metric_exploration.Metrics_Models(df,non_metric_columns,categories)

In [276]:
scores = test.run_model(model_type = "MLP")

(1332, 13)
(1332,)
started training epoch no. 1


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File &quot;/home/shaul/workspace/GitHub/SOTA/venv_sota/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py&quot;, line 178, in _worker_loop
    data = fetcher.fetch(index)
  File &quot;/home/shaul/workspace/GitHub/SOTA/venv_sota/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py&quot;, line 47, in fetch
    return self.collate_fn(data)
  File &quot;/home/shaul/workspace/GitHub/SOTA/venv_sota/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py&quot;, line 79, in default_collate
    return [default_collate(samples) for samples in transposed]
  File &quot;/home/shaul/workspace/GitHub/SOTA/venv_sota/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py&quot;, line 79, in &lt;listcomp&gt;
    return [default_collate(samples) for samples in transposed]
  File &quot;/home/shaul/workspace/GitHub/SOTA/venv_sota/lib/python3.6/site-packages/torch/utils/

In [266]:
scores.keys()

dict_keys([&#39;dataset&#39;, &#39;random&#39;, &#39;radical_or_centralist&#39;, &#39;fi_label_combined&#39;, &#39;fi_reduced_label_combined&#39;])